<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/main/Capstone_Project_Group8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project: Questionnaires

In [7]:
import json
import time
import google.generativeai as genai
from google.colab import userdata
import requests

key = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=key)

ai_model = genai.GenerativeModel("gemini-1.5-flash")

url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/questionnaires/questionnaire1.json'

response = requests.get(url)

if response.status_code == 200:
    try:
        data1 = response.json()
    except json.JSONDecodeError:
        print("Fehler beim Parsen der JSON-Daten.")
else:
    print(f"Fehler: Statuscode {response.status_code}")

print(data1)

[{'id': 'aa2d8cdd-0758-4035-b0b6-ca18e2f380d8', 'type': 'SINGLE_SELECT', 'question': 'Data processing consent', 'options': [{'id': 'ee0437c0-6335-4b88-8bc5-d4eb8e2c68bf', 'option': 'Yes'}, {'id': 'd357ab84-929f-440a-b9ad-42ff36402a53', 'option': 'No'}]}, {'id': '12e1ed1d-edaa-4e93-8645-de3850e998f9', 'type': 'SINGLE_SELECT', 'question': 'Customer group', 'options': [{'id': '53cc44fa-397b-40bf-8af8-3f629818c93b', 'option': 'End User'}, {'id': 'bab850d1-2de8-4cae-9f87-2c86d41615e8', 'option': 'Wholesaler, Distributor'}, {'id': '31ffae1e-f742-4a0b-b279-eb393483f075', 'option': 'Consultant, Planner, Architect'}, {'id': 'ed56f777-a7cd-48db-9d36-e2579c57e361', 'option': 'R&D'}]}, {'id': '625012ae-9192-4cf6-a73d-55e1813d6014', 'type': 'MULTI_SELECT', 'question': 'Products interested in', 'options': [{'id': '4ccbe8ae-ce37-4822-830a-542aa0475d30', 'option': 'MY-SYSTEM'}, {'id': '58fdea0d-46db-41ef-a089-825f6ebfbb99', 'option': 'Notion'}, {'id': '13906f18-4e39-4d39-8da9-b5e49d9e4cc5', 'option': 

In [8]:
def process_json(data):
    """Verarbeitet die JSON-Daten und fügt Antworten hinzu."""
    for item in data:
        if 'options' in item:
            for option in item['options']:
                response_text = generate_answer(item['question'], option['option'])
                if response_text:
                    answer_list = [part.strip() for part in response_text.split("§")]
                    option['text'] = answer_list
                    print(f"Antworten generiert für Frage {item['question']} und Antwort {option['option']}")
                    time.sleep(3)
    return data


def generate_answer(question, option):
    """Generiert eine Antwort auf eine Frage basierend auf der Option."""
    prompt = f"""
    You are the user of an app and you are responding in a spoken style to the following question.
    You like to talk so you don't just say yes or no but rather answer with a whole sentence.
    Question: "{question}"
    Your answer should contain following content (e.g. if the content of the answer is yes, you convey this in your response):
    Content of the answer: "{option}"
    The responses should be in the following format and be kind of random so that each answer is in a different in style.
    Generate 5 answers that are split by a § sign and contain only text.
    answer1§answer2§...§answer5
    """
    try:
        response = ai_model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Fehler bei der Generierung: {e}")
        return None

In [9]:
processed_data = process_json(data1)

Antworten generiert für Frage Data processing consent und Antwort Yes
Antworten generiert für Frage Data processing consent und Antwort No
Antworten generiert für Frage Customer group und Antwort End User
Antworten generiert für Frage Customer group und Antwort Wholesaler, Distributor
Antworten generiert für Frage Customer group und Antwort Consultant, Planner, Architect
Antworten generiert für Frage Customer group und Antwort R&D
Antworten generiert für Frage Products interested in und Antwort MY-SYSTEM
Antworten generiert für Frage Products interested in und Antwort Notion
Antworten generiert für Frage Products interested in und Antwort JTS
Antworten generiert für Frage Products interested in und Antwort JS EcoLine
Antworten generiert für Frage Products interested in und Antwort AKW100
Antworten generiert für Frage Products interested in und Antwort AX100
Antworten generiert für Frage What kind of follow up is planned und Antwort Email
Antworten generiert für Frage What kind of follo

In [10]:
with open('q1_processed.json', 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)